# Chapter 3: Plot your results and upload them 

In this unit, you will learn how to plot the observations recorded through the experiment.<br>
You will also learn to upload your experiments to the OpenAI webpage for everyone to compare their algorithms to yours.

## Plot Results from Monitor

Launch a learning script, for instance, the <b>simple_qlearn.py</b>.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun gym_construct simple_qlearn.py

Once launched, it will generate the monitor data in the path = **outdir** = **'/tmp/gazebo_gym_experiments'**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd /tmp/gazebo_gym_experiments
ls

You should get the following experiment monitor files:

**openaigym.episode_batch.xxxx.stats.json**<br>
**openaigym.manifest.xxxx.manifest.json**

So now, let's plot it. For that, just execute the following script:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun gym_construct plot_gazebo_gym_experiments.py

This script will save a copy of the plot data in the path <b>/tmp/experiment_data_plot.png</b>, and open a plot window in the Graphical Interface window. To open the Graphical Interface window, just click on the following button, on the upper right-hand corner of the IDE:

<img src="img/graphical_interface.png"/>

You should see the episodes' data scores plotted like so:

<img src="img/plot_example.png"/>

Take a look at the code inside:

<p style="background:green;color:white;">**plot_gazebo_gym_experiments.py**</p>

In [1]:
#!/usr/bin/env python
import sys
print sys.path
#sys.path.append("/usr/local/lib/python2.7/dist-packages/")
#sys.path.remove("/usr/lib/python2.7/dist-packages")
import numpy as np
print np.__file__
print np.__version__
import os
import gym

import matplotlib
import matplotlib.pyplot as plt
import itertools
import sys
import argparse

from scipy.interpolate import pchip

class LivePlot(object):
    def __init__(self, outdir, data_key='episode_rewards', line_color='blue'):
        """
        Liveplot renders a graph of either episode_rewards or episode_lengths
        Args:
            outdir (outdir): Monitor output file location used to populate the graph
            data_key (Optional[str]): The key in the json to graph (episode_rewards or episode_lengths).
            line_color (Optional[dict]): Color of the plot.
        """
        #data_key can be set to 'episode_lengths'
        self.outdir = outdir
        self._last_data = None
        self.data_key = data_key
        self.line_color = line_color

        #styling options
        matplotlib.rcParams['toolbar'] = 'None'
        plt.style.use('ggplot')
        plt.xlabel("episodes")
        plt.ylabel("cumulated episode rewards")
        fig = plt.gcf().canvas.set_window_title('averaged_simulation_graph')
        matplotlib.rcParams.update({'font.size': 15})

    def plot(self, full=True, dots=False, average=0, interpolated=0):
        results = gym.monitoring.load_results(self.outdir)
        data =  results[self.data_key]
        avg_data = []

        if full:
            plt.plot(data, color='blue')
        if dots:
            plt.plot(data, '.', color='black')
        if average > 0:
            average = int(average)
            for i, val in enumerate(data):
                if i%average==0:
                    if (i+average) < len(data)+average:
                        avg =  sum(data[i:i+average])/average
                        avg_data.append(avg)
            new_data = expand(avg_data,average)
            plt.plot(new_data, color='red', linewidth=2.5) 
        if interpolated > 0:
            avg_data = []
            avg_data_points = []
            n = len(data)/interpolated
            if n == 0:
                n = 1
            data_fix = 0
            for i, val in enumerate(data):
                if i%n==0:
                    if (i+n) <= len(data)+n:
                        avg =  sum(data[i:i+n])/n
                        avg_data.append(avg)
                        avg_data_points.append(i)
                if (i+n) == len(data):
                    data_fix = n
            
            x = np.arange(len(avg_data))
            y = np.array(avg_data)
            #print x
            #print y
            #print str(len(avg_data)*n)
            #print data_fix
            interp = pchip(avg_data_points, avg_data)
            xx = np.linspace(0, len(data)-data_fix, 1000)
            plt.plot(xx, interp(xx), color='green', linewidth=3.5)        

        # pause so matplotlib will display
        # may want to figure out matplotlib animation or use a different library in the future
        # plt.pause(0.000001)

def expand(lst, n):
    lst = [[i]*n for i in lst]
    lst = list(itertools.chain.from_iterable(lst))
    return lst

def pause():
    programPause = raw_input("Press the <ENTER> key to finish...")

if __name__ == '__main__':
    
    outdir = '/tmp/gazebo_gym_experiments'
    plotter = LivePlot(outdir)

    parser = argparse.ArgumentParser()
    parser.add_argument("-f", "--full", action='store_true', help="print the full data plot with lines")
    parser.add_argument("-d", "--dots", action='store_true', help="print the full data plot with dots")
    parser.add_argument("-a", "--average", type=int, nargs='?', const=50, metavar="N", help="plot an averaged graph using N as average size delimiter. Default = 50")
    parser.add_argument("-i", "--interpolated", type=int, nargs='?', const=50, metavar="M", help="plot an interpolated graph using M as interpolation amount. Default = 50")
    args = parser.parse_args()

    if len(sys.argv)==1:
        # When no arguments are given, plot the full data graph
        plotter.plot(full=True)
    else:
        plotter.plot(full=args.full, dots=args.dots, average=args.average, interpolated=args.interpolated)

    plt_save_path = "/tmp/experiment_data_plot.png"
    plt.savefig(plt_save_path)
    print ("Saved plot in "+plt_save_path)
    print "Showing Plot in RemoteScreen"
    plt.show()
    # pause()


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-1c540af380e0>, line 3)

<p style="background:green;color:white;">**plot_gazebo_gym_experiments.py**</p>

All of the code is quite self-explanatory. You can modify some plotting options and change, of course, the source of the data, as well as the place to save them.<br>
As you can see, you will need a large quantity of episodes to see if your learning algorithm is really working.

<p style="background:#EE9023;color:white;">**Exercise 3.1**</p>

Make your own version of a plot. For that, retrieve the original file and copy it to your workspace.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd gym_construct/scripts
cp plot_gazebo_gym_experiments.py /home/user/catkin_ws/src/

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
python /home/user/catkin_ws/src/plot_gazebo_gym_experiments.py

This way, you will be able to edit the file and work with it in the IDE without any issue. You can save the image in the path **/home/user/catkin_ws/src**. This way, you will be able to visualize it directly in the IDE. If not, you can also use this command to open it in an image editor and visualize it in the Graphical Interface:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
eog path_to_your_image_file

## Upload Results from Monitor

<p style="background:#CD2122;color:white;">**WARNING**</p><br>
**The upload for the Gym-Gazebo module is not yet supported by Gym**. However, we think it is necessary to explain how it would be done in the basic Gym module because it's a neat feature that we hope will also be available for Gym-Gazebo in the near future.

So you won't be able to do this with this Turtlebot Gazebo simulation.

<p style="background:#CD2122;color:white;">**END of WARNING**</p>

To upload your experiment to the OpenAI Web, you have to follow these steps:

#### 1. **Get the API Key**

Click the API key link at the end of the example section in the README file:
https://github.com/openai/gym/blob/master/README.rst<br>

Which is this direct link here :<br>
https://gym.openai.com/settings/profile<br><br>
Accept the terms (you accept by giving your email address) and it will take you to your user's space :<br>
https://gym.openai.com/users/YOUR_USERNAME

You will get the API_Key on the top right-hand corner.

#### 2. **Execute code**

Execute the learning code in order to generate all of the monitor data.

#### 3. **Upload it through the Gym API**

Just execute this code in this same notebook, through the Python notebook, or execute a Python script with this code inside:

In [ ]:
import gym
gym.upload('/tmp/gazebo_gym_experiments', api_key='YOUR_API_KEY')

You get a link :  https://gym.openai.com/evaluations/SOME_NUMBER_HERE

To appear in your Zone, you have to execute a minimum of 100 episodes
https://gym.openai.com/users/YOUR_USERNAME